In [68]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import geopandas as gpd 
import matplotlib.pyplot as plt
# import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString

In [69]:


deaths = gpd.read_file("../migrant_deaths_analysis/death_files/ogis_migrant_deaths_091021_with_yuma.geojson")
deaths=deaths[deaths['Latitude'].notnull()]
deaths['Latitude']= deaths.Latitude.astype(float)
deaths['Longitude']= deaths.Longitude.astype(float)
deaths = gpd.GeoDataFrame(deaths, geometry=gpd.points_from_xy(deaths.Longitude,deaths.Latitude))

deaths = deaths.set_crs('epsg:4326')
# deaths = deaths.to_crs('epsg:3857')


deaths['Reporting Date']= pd.to_datetime(deaths['Reporting Date'])
deaths['Reporting_year']= pd.to_datetime(deaths['Reporting Date']).dt.year



print(deaths.total_bounds)

print(deaths.total_bounds[0])
print(deaths.total_bounds[1])


minx, miny, maxx, maxy = deaths.total_bounds



[-114.80891    31.33271  -109.053383   33.94276 ]
-114.80891
31.33271


In [70]:
import requests
import urllib
import codecs
import json
from osm2geojson import json2geojson

OVERPASS = "https://overpass-api.de/api/interpreter/"


def overpass_call(query):
    encoded = urllib.parse.quote(query.encode('utf-8'), safe='~()*!.\'')
    r = requests.post(OVERPASS,
                      data=f"data={encoded}",
                      headers={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'})
    if not r.status_code is 200:
        raise requests.exceptions.HTTPError('Overpass server respond with status '+str(r.status_code))
    return r.text


def save_data(data, geom_file):
    json_data = json.dumps(data, indent=2)
    f = codecs.open(geom_file, 'w')
    f.write(json_data)
    f.close()


data = overpass_call(f"""
    [out:json][timeout:55];
    (
    node["place"="city"]({miny},{minx},{maxy},{maxx});
    way["place"="city"][type="boundary"]({miny},{minx},{maxy},{maxx});
    relation["place"="city"][type="boundary"]({miny},{minx},{maxy},{maxx});
    


    
    node["place"="town"]({miny},{minx},{maxy},{maxx});
    way["place"="town"][type="boundary"]({miny},{minx},{maxy},{maxx});
    relation["place"="town"][type="boundary"]({miny},{minx},{maxy},{maxx});
    


    
    )
    ;out geom;
""")

geojson_data = json2geojson(data)
save_data(geojson_data, '../migrant_deaths_analysis/distance_from_roads_towns_border/towns/cities_towns.geojson')




dfff = gpd.read_file('../migrant_deaths_analysis/distance_from_roads_towns_border/towns/cities_towns.geojson', driver='GeoJSON')
dfff = dfff.set_crs('epsg:4326')


countries = gpd.read_file('../migrant_deaths_analysis/distance_from_roads_towns_border/towns/countries.geojson', driver='GeoJSON')
countries = countries.set_crs('epsg:4326')
countries = countries[countries["ADMIN"]=="United States of America"]  

dfff = gpd.sjoin(dfff,countries, how="inner", op='intersects')

dfff = dfff.to_crs('epsg:26912')
# dff = dff.to_crs('epsg:3857')

tags = dfff['tags'].apply(pd.Series)


fdd= dfff.merge(tags, left_index=True, right_index=True)

fdd= fdd.sort_values('type_x').drop_duplicates(subset=['name'], keep='last')

fdd.to_file('../migrant_deaths_analysis/distance_from_roads_towns_border/towns/cities_towns.geojson', driver='GeoJSON')
fdd



multipolygon member not handled {'type': 'node', 'ref': 150975216, 'role': 'admin_centre', 'lat': 31.5455709, 'lon': -110.2772249}
multipolygon member not handled {'type': 'node', 'ref': 1340670207, 'role': 'admin_centre', 'lat': 33.5806115, 'lon': -112.237294}
multipolygon member not handled {'type': 'node', 'ref': 150935721, 'role': 'admin_centre', 'lat': 33.3959623, 'lon': -110.7871344}
multipolygon member not handled {'type': 'node', 'ref': 150981213, 'role': 'admin_centre', 'lat': 32.9875659, 'lon': -110.7709387}
multipolygon member not handled {'type': 'node', 'ref': 150939528, 'role': 'label', 'lat': 32.1994951, 'lon': -110.9683439}
multipolygon member not handled {'type': 'node', 'ref': 150964034, 'role': 'admin_centre', 'lat': 32.2228765, 'lon': -110.9748477}
Invalid geometry! Try to fix with 0 buffer
Geometry fixed!
multipolygon member not handled {'type': 'node', 'ref': 151639300, 'role': 'admin_centre', 'lat': 32.8961755, 'lon': -109.8275815}
multipolygon member not handled

ERROR:fiona._env:cities_towns.geojson: No such file or directory


,type_x,id,tags,geometry,index_right,ADMIN,ISO_A3,name,place,population,wikidata,wikipedia,census:population,ele,gnis:County,gnis:County_num,gnis:ST_alpha,gnis:ST_num,gnis:id,import_uuid,is_in,gnis:Class,name:uk,gnis:feature_id,population:date,website,name:en,name:ru,population:source,source,alt_name,official_name,founded,incorporated,start_date,name:ar,name:he,alt_name:es,capital,layer,name:da,name:eo,name:es,name:hak,name:haw,name:hi,name:ja,name:kn,name:ko,name:lt,name:lv,name:nan,name:nv,name:oc,name:pl,name:pt,name:ta,name:zh,name:zh-Hans,name:zh-Hant,source:name:oc,state_capital,fixme,addr:postcode,admin_level,border_type,boundary,is_in:country,is_in:country_code,is_in:iso_3166_2,is_in:state,is_in:state_code,tiger:CLASSFP,tiger:CPI,tiger:FUNCSTAT,tiger:LSAD,tiger:MTFCC,tiger:NAME,tiger:NAMELSAD,tiger:PCICBSA,tiger:PCINECTA,tiger:PLACEFP,tiger:PLACENS,tiger:PLCIDFP,tiger:STATEFP,tiger:reviewed,type_y
29,node,150974590,"{'ele': '348', 'gnis:id': '11953', 'name': 'Su...",POINT (381100.176 3721759.422),238,United States of America,USA,Sun City,town,37499,Q1006586,"en:Sun City, Arizona",NaN,348,NaN,NaN,NaN,NaN,11953,NaN,NaN,NaN,Сан-Сіті,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,node,150976539,"{'ele': '426', 'gnis:id': '24269', 'name': 'Ca...",POINT (429154.465 3638182.873),238,United States of America,USA,Casa Grande,town,NaN,Q79889,"en:Casa Grande, Arizona",NaN,426,NaN,NaN,NaN,NaN,24269,NaN,NaN,NaN,Каза-Ґранде,NaN,NaN,NaN,NaN,Каса-Гранде,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,node,150981505,"{'ele': '364', 'gnis:id': '25113', 'name': 'Su...",POINT (418416.306 3675038.203),238,United States of America,USA,Sun Lakes,town,NaN,NaN,NaN,NaN,364,NaN,NaN,NaN,NaN,25113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,node,151408617,"{'alt_name': 'City of Bisbee', 'ele': '1688', ...",POINT (601821.805 3479767.273),238,United States of America,USA,Bisbee,town,NaN,NaN,NaN,NaN,1688,NaN,NaN,NaN,NaN,1436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,City of Bisbee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,node,151939670,"{'ele': '1447', 'gnis:Class': 'Populated Place...",POINT (656298.428 3658218.428),238,United States of America,USA,Morenci,town,NaN,NaN,NaN,NaN,1447,Greenlee,011,AZ,04,32021,bb7269ee-502a-5391-8056-e3ce0e66489c,"Greenlee,Arizona,Ariz.,AZ,USA",Populated Place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,node,150971258,"{'ele': '358', 'gnis:id': '12009', 'name': 'Su...",POINT (373115.486 3721885.656),238,United States of America,USA,Surprise,city,117517,Q51687,"en:Surprise, Arizona",NaN,358,NaN,NaN,NaN,NaN,12009,NaN,NaN,NaN,Серпрайз,NaN,2010-04-01,NaN,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,node,335576581,"{'ele': '295', 'fixme': 'Move closer to center...",POINT (373800.459 3700377.383),238,U

In [71]:
fdd['name'].value_counts(dropna=False).to_frame().to_csv('../migrant_deaths_analysis/distance_from_roads_towns_border/towns/cities_towns_name.csv')


In [72]:
print(gpd.__version__)


0.9.0


In [73]:
# deaths = deaths.reset_index()
print(dfff.crs )
print(deaths.crs )
# deaths = deaths.to_crs('epsg:3857')
deaths = deaths.to_crs('epsg:26912')
# fdd = fdd.to_crs('epsg:3857')

fdd = fdd.to_crs('epsg:26912')

print(fdd.crs )
print(deaths.crs )

epsg:26912
epsg:4326
epsg:26912
epsg:26912


In [74]:

# distances1 = 
# min_idx1 = distances1.idxmin()


# distances1

def calculate_nearest(row, lines):
    
#     print(row['index'])
    distances1 = fdd.distance(row['geometry'])
    
    min_val=distances1.min()
    print(min_val)
    min_indices=[idx for idx,val in distances1.items() if val==min_val]
#     print(min_indices)
    
    if len(min_indices)>1:
        print('more than one min')
        print(len(min_indices))
        print('done')
    
    mins_list = []
    for mins in min_indices:
        border_seg1 = lines['id'][mins]
        mins_list.append(border_seg1)
#     print(mins_list)
    return [min_val,mins_list]

    
#     print( row['geometry'][0])

deaths['town_distance'] = deaths.apply(lambda row : calculate_nearest(row, fdd),axis=1)
                                      
                                      
                                      
                                      
towns = deaths.town_distance.apply(pd.Series)
towns.columns = ['distance', 'IDs']                                       





7381.303872986647
22582.678623804346
25608.50391883214
4257.457735897566
23848.199405922387
23827.781449349335
23835.172548018385
23855.58419371261
35895.3422654162
0.0
26380.169474548515
2357.108295805869
26673.73489544713
44070.14184624271
53696.89747140682
0.0
16526.54763889956
2131.8780754308063
39260.7132782134
0.0
8347.206599576868
14174.668097484433
4259.494416856484
0.0
25574.90523144794
41914.857757242724
45493.63861986369
12981.835000876645
10599.089593084496
0.0
17435.268663025974
7869.169063710572
25441.72077323384
7081.898380976869
24448.093029041498
18888.24647930609
7674.318383953126
31706.85199555197
44505.477292204625
0.0
0.0
50613.483472746055
51373.76571807908
65903.27795212717
44579.42821411659
0.0
43066.26123531783
2067.513048219577
51362.0379965017
34243.67844809559
30929.23584579943
37004.903686182806
0.0
32397.123446639358
0.0
18466.379491888667
7290.148607864933
7278.160855931787
7294.072381132897
11079.85298972764
0.0
0.0
12093.569175673521
8849.252416019588
2

13932.586973924666
26930.02886026799
10134.91010925071
12277.765351652137
27157.36574617287
16019.5353549761
11808.876238366036
0.0
28356.253267738175
10954.463003342373
29920.06906762323
7204.718924730102
34296.095597531144
9122.190474860414
22010.37855061052
22003.535197924997
22010.412818127177
17256.507704158314
20467.73257117988
393.2224825406312
21238.025841968512
0.0
28802.24664889146
18520.760822966535
37489.07512893041
47946.77772945789
19392.649857493358
19402.956666565686
33675.69879734821
24016.32785835053
29300.607126697578
2265.4906483588998
11704.08319904729
11726.052564073525
10355.257896281239
35691.91270092922
35670.06969236971
15854.684213934876
16462.871214852763
31370.127789867485
11851.062516214946
14464.942310791332
53148.01529670759
26843.138072791844
23086.241932879093
14615.041840791273
8678.939238144463
19489.654763064977
36243.738966719895
36240.60713214464
22707.73053193479
24453.928646668875
26162.67597112344
29684.057730781573
34463.67323084803
28831.0385

8273.304683411814
8266.007662999804
25171.664618345392
40565.71546635631
0.0
10589.422445857239
0.0
28034.077672649273
28020.44110541096
28019.28521779581
22807.048682122193
25477.76200677005
11034.973915331317
40338.07269932761
17493.619853700224
13677.533715673935
13481.468512329779
38783.63772791981
1814.4984047959254
1723.0386923851286
1560.7062318280268
31956.337627618064
49037.74548907003
34259.23648516468
25325.209009679256
10312.091443599531
47118.03793873475
21652.09784988275
42424.77803568739
23203.659940515456
23203.423329482164
22160.30830781648
22292.93626440207
26025.036519341455
26028.61479594266
24763.12427357374
23929.025874339677
4072.2321231502733
39213.847459911995
24879.213121448283
0.0
27430.591337197882
41067.93444131409
308.21284576603676
39040.2695117938
6161.18992381269
28006.295755077746
20802.95225021692
8278.698420702109
37153.87029474076
26857.048169990318
28704.63638566606
15037.2874056164
24468.355552196725
24562.27054319274
27798.08863573435
35804.88506

66111.3862081696
35109.265152463806
23776.661378715824
3156.8149162854684
16276.291322010393
35482.968884802525
19053.976072418238
19769.56688839057
20789.6561018114
25545.428477341236
33082.71159038296
33728.00596692219
31248.79837505253
24211.73629595006
25159.899462628593
19825.042258865873
12423.174669540482
37295.26524426308
12641.302035723866
34515.91567632137
33040.132231756084
52684.53006529791
52662.65116733353
36944.819886967765
35946.21328835223
43835.95297257359
11892.000465655992
36199.52580134105
0.0
12563.23412226189
9260.051048723104
8047.470532661615
7361.913055208858
25197.945383772323
35471.95706538677
49461.83785288583
57069.499931412705
32460.001815830405
15186.7754916141
34068.73682026451
15065.340732003599
50788.08241306586
27833.785180480765
26083.69675423753
1624.3965183010266
1705.3074598744836
8760.1657411668
24614.781124909918
18281.516005850077
13651.037992151432
17201.835805078794
19754.79309028648
34234.15033512746
35001.40497647757
33012.68537291139
3303

39271.85432732397
16839.872146822898
26399.250896380992
26379.68800340229
45962.48990363126
22913.42737179803
32186.60247254672
7754.795042854508
38956.88860310335
5363.785403612351
20930.971440118705
26949.305482641594
46086.67087776967
7832.2919895759505
38578.50691315748
49838.005126469005
29601.922791681587
48545.02195822919
29077.8721989984
42702.93562881131
51248.218182066936
24309.538779590865
49812.140750720246
26817.466172962406
46728.41652482774
22357.251267277512
22169.132310401037
6486.516445399086
32671.912915062043
18013.92866279564
40982.786458945804
22153.699844166636
3975.1401482120336
30872.294445617285
36145.83039190371
37855.03066900472
9831.331398018025
51940.817689960335
9801.527563970154
32250.52448328841
0.0
0.0
46337.18416530257
17170.04004391001
49955.84109654397
10819.949215358622
14882.170626791698
23713.91472217883
0.0
17368.084697210237
56731.901532660646
0.0
32158.89515478471
33270.09590049393
8348.094259277099
22169.996010313193
23444.558597912408
16538.

36050.98791167449
8333.090491397174
42720.68390302282
16817.348703917338
42470.17700109001
42458.37977691592
42454.204189512
37215.59648604951
33968.78508990978
11632.835606679631
14298.547256611591
15245.513826809378
25129.469127137807
33836.53678779623
28173.14189724319
31450.583083777117
0.0
10976.888870614699
18674.830155544903
16391.313732690953
24388.97368879056
23960.098225338654
42876.43236122587
23248.259571601167
37772.72542878046
50630.64232586319
62980.89700972633
38315.897164134345
23260.03849871873
23255.227368312528
32085.546121137297
23031.68501386746
31969.546811208045
42918.9999813073
29479.745856092246
22754.959114358044
15310.737555029087
22739.200613361074
39154.83494459638
19796.215035771904
57684.93851001612
50743.06780279755
19979.01015094526
24949.194243984475
29665.82771087752
22848.88489079991
16581.216272132737
0.0
917.3969995172139
24017.63502556575
42731.49853674659
29025.234957005025
8740.581089129852
32012.0940235407
28958.780459632122
30585.911972043974

24976.349059517874
27452.981089588407
22276.882232884363
33591.962191401806
36033.05353062043
28277.46170301524
49208.39705472416
14319.902145467513
11634.866556113368
19382.092092873543
26110.78696870265
40372.315618410044
16324.71471139368
53361.37886119443
35994.58433993795
0.0
35107.30781534922
21658.872915661243
27736.284015497444
35820.28917586959
38274.51806084221
55933.40292203095
33665.96916834657
14546.113504029798
42685.57101761887
27706.451460569337
17969.506016455645
35112.769668455214
11739.247156681819
29623.735412089758
25599.8755222399
26835.066758692526
526.5721802673656
29239.16785853436
27224.039235041255
23377.787908905717
6697.390194704987
33493.84557414319
24420.688587879227
36850.39083541149
40025.5738584023
38859.80249639214
33653.54815431281
35644.76723849633
8454.862355910922
23825.508537298963
6718.662700036725
15962.610604105072
6807.439961854499
22509.61899239624
52890.643758017504
26059.116806847673
20602.278856178207
136.13437076909935
3218.447431683231


23607.067078403907
28327.812760888355
5935.347770628505
19649.388389769018
23477.96099471695
28878.591576968724
15182.119652158213
50217.162684550574
46169.15380646723
35759.186501091295
4419.875003721087
4407.371733027753
52609.33816841924
28766.00296095501
32390.584833929523
21888.493492520553
50438.71933414023
40259.22851828057
42561.640749814425
28225.550765484088
27182.271956589186
53139.857803325875
25076.188318843757
25256.41312260068
27506.735860578774
39508.866298853005
26758.87941225253
37161.98990409263
17051.167472315454
0.0
7997.324172718752
1836.9229882049567
7079.489219907349
5199.2164668530795
493.0976002567504
0.0
0.0
0.0
0.0
16967.020937998455
4870.935002588049
16861.363801472675
0.0
3637.6596250342072
0.0
7724.710119860676
0.0
53071.46670543124
39538.805002768866
11548.38614619672
14584.759290191083
12053.987257755623
0.0
0.0
62351.591377410856
37895.82176145238
29313.96559960987
10979.286518375175
0.0
32563.980582222102
32563.980582222102
4588.3029689147625
52878.41

25263.339585701135
50541.595668509624
50597.52480281542
1363.2693218550255
2081.975031902338
1289.4338200081506
5779.980952573416


In [75]:


# min_val=min(my_list)
# min_indices=[idx for idx,val in enumerate(my_list) if val==min_val]

deaths_towns = towns.merge(deaths,left_index=True, right_index=True)
deaths_towns

# border_seg1 = border_lines['geometry'][min_idx1]







# deaths

,distance,IDs,ML Number,Name,Sex,Age,Reporting Date,Surface Management,Location,Location Precision,Corridor Code,Corridor,Cause of Death,OME Determined COD,Condition Code,Body Condition,Post Mortem Interval,State,County,Latitude,Longitude,UTM X,UTM Y,Date,OME number,YCSO number,Nationality,merge_id,geometry,Reporting_year,road_distance
0,7381.303873,[2590624],00-00097,Unidentified,male,,2000-01-10,Cocopah Indian Reservation,,"physical description with directions, distance...",12,Yuma,Skeletal Remains,UNDETERMINED - MUMMIFIED AND SKELETAL REMAINS,4,Skeletonization w/ mummification,< 5 weeks,Arizona,YUMA,32.589668,-114.799807,143294,3612175,2000-01-10T00:00:00,None,None,None,00-00097,POINT (143293.359 3612175.618),2000.0,"[7381.303872986647, [2590624]]"
1,22582.678624,[110679],00-00148,"Nicolas, Tomas",male,17,2000-01-22,State of Arizona,,"physical description with directions, distance...",4,Patagonia,Exposure,HYPOTHERMIA,1,Fully fleshed,< 1 day,Arizona,PIMA,31.812406,-110.483205,548914,3519758,2000-01-22T00:00:00,None,None,None,00-00148,POINT (548913.472 3519758.907),2000.0,"[22582.678623804346, [110679]]"
2,25608.503919,[5606091],00-00189,Unidentified,male,,2000-01-20,US Forest Service,,"physical description with directions, distance...",5,Nogales,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,SANTA CRUZ,31.568000,-111.151300,485643,3492563,2000-01-20T00:00:00,None,None,None,00-00189,POINT (485642.300 3492563.305),2000.0,"[25608.50391883214, [5606091]]"
3,4257.457736,[150963593],00-00222,Unidentified,male,,2000-01-10,Private,,"physical description with directions, distance...",9,Ajo,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,6,Complete skeletonization w/ disarticulation,< 6-8 months,Arizona,PIMA,32.355997,-112.826855,328102,3581363,2000-01-10T00:00:00,None,None,None,00-00222,POINT (328101.292 3581363.326),2000.0,"[4257.457735897566, [150963593]]"
4,23848.199406,[150943192],00-00245,"Candia-Bravo, Maria Del Rocio",female,20,2000-02-05,Tohono Oodham Nation,,"physical description with directions, distance...",7,San Miguel,Blunt Force Injury,MULTIPLE BLUNT FORCE INJURIES,1,Fully fleshed,< 1 day,Arizona,PIMA,31.717790,-111.779830,426116,3509419,2000-02-05T00:00:00,None,None,None,00-00245,POINT (426115.142 3509419.894),2000.0,"[23848.199405922387, [150943192]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3909,50597.524803,[150963593],None,None,Male,22,2021-08-31,Cabeza Prieta,None,None,None,None,None,UNDETERMINED - SKELETAL REMAINS,None,None,None,None,None,32.363920,-113.401930,None,None,2021-08-31T00:00:00,2021-1141,2021-70152,Guatemala,2021-1141,POINT (273997.077 3583311.350),2021.0,"[50597.52480281542, [150963593]]"
3910,1363.269322,[2590624],None,None,Female,37,2021-09-10,Yuma,None,None,None,None,None,Exposure to High Outdoor Temperature,None,None,None,None,None,32.460636,-114.616562,None,None,2021-09-10T00:00:00,2021-1182,2021-73187,Mexican,2021-1182,POINT (160018.874 3597259.248),2021.0,"[1363.2693218550255, [2590624]]"
3911,2081.975032,[2590624],None,None,Female,42,2021-09-10,Yuma,None,None,None,None,None,Exposure to High Outdoor Temperatures,None,None,None,None,None,32.472349,-114.608894,None,None,2021-09-10T00:00:00,2021-1183,2021-73259,Mexican,2021-1183,POINT (160784.122 3598534.300),2021.0,"[2081.975031902338, [2590624]]"
3912,1289.433820,[2590624],None,None,Male,33,2021-09-10,Yuma,None,None,None,None,None,Exposure to High Outdoor Temperatures,None,None,None,None,None,32.478630,-114.617300,None,None,2021-09-10T00:00:00,2021-73360,2021-73360,Mexican,2021-73360,POINT (160017.212 3599257.947),2021.0,"[1289.4338200081506, [2590624]]"


In [76]:


def unpack(row):
    if len(row['IDs']) < 1:
        return None
    else:
        return row['IDs'][0]

def unpack2(row):
    if len(row['IDs']) < 2:
        return None
    else:
        return row['IDs'][1]
    

deaths_towns['unpacked_ID'] = deaths_towns.apply(lambda row : unpack(row),axis=1)

deaths_towns['unpacked_ID_2'] = deaths_towns.apply(lambda row : unpack2(row),axis=1)

deaths_towns_merged = deaths_towns.merge(fdd, left_on="unpacked_ID",right_on="id",how='left')


deaths_towns_merged

,distance,IDs,ML Number,Name,Sex,Age,Reporting Date,Surface Management,Location,Location Precision,Corridor Code,Corridor,Cause of Death,OME Determined COD,Condition Code,Body Condition,Post Mortem Interval,State,County,Latitude,Longitude,UTM X,UTM Y,Date,OME number,YCSO number,Nationality,merge_id,geometry_x,Reporting_year,road_distance,unpacked_ID,unpacked_ID_2,type_x,id,tags,geometry_y,index_right,ADMIN,ISO_A3,name,place,population,wikidata,wikipedia,census:population,ele,gnis:County,gnis:County_num,gnis:ST_alpha,gnis:ST_num,gnis:id,import_uuid,is_in,gnis:Class,name:uk,gnis:feature_id,population:date,website,name:en,name:ru,population:source,source,alt_name,official_name,founded,incorporated,start_date,name:ar,name:he,alt_name:es,capital,layer,name:da,name:eo,name:es,name:hak,name:haw,name:hi,name:ja,name:kn,name:ko,name:lt,name:lv,name:nan,name:nv,name:oc,name:pl,name:pt,name:ta,name:zh,name:zh-Hans,name:zh-Hant,source:name:oc,state_capital,fixme,addr:postcode,admin_level,border_type,boundary,is_in:country,is_in:country_code,is_in:iso_3166_2,is_in:state,is_in:state_code,tiger:CLASSFP,tiger:CPI,tiger:FUNCSTAT,tiger:LSAD,tiger:MTFCC,tiger:NAME,tiger:NAMELSAD,tiger:PCICBSA,tiger:PCINECTA,tiger:PLACEFP,tiger:PLACENS,tiger:PLCIDFP,tiger:STATEFP,tiger:reviewed,type_y
0,7381.303873,[2590624],00-00097,Unidentified,male,,2000-01-10,Cocopah Indian Reservation,,"physical description with directions, distance...",12,Yuma,Skeletal Remains,UNDETERMINED - MUMMIFIED AND SKELETAL REMAINS,4,Skeletonization w/ mummification,< 5 weeks,Arizona,YUMA,32.589668,-114.799807,143294,3612175,2000-01-10T00:00:00,None,None,None,00-00097,POINT (143293.359 3612175.618),2000.0,"[7381.303872986647, [2590624]]",2590624,None,relation,2590624,"{'admin_level': '8', 'border_type': 'city', 'b...","MULTIPOLYGON (((141619.197 3601615.589, 141590...",238,United States of America,USA,San Luis,city,NaN,Q80018,"en:San Luis, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"USA, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,city,administrative,USA,US,US:AZ,Arizona,AZ,C1,Y,A,25,G4110,San Luis,San Luis city,N,N,63470,02411795,0463470,04,no,boundary
1,22582.678624,[110679],00-00148,"Nicolas, Tomas",male,17,2000-01-22,State of Arizona,,"physical description with directions, distance...",4,Patagonia,Exposure,HYPOTHERMIA,1,Fully fleshed,< 1 day,Arizona,PIMA,31.812406,-110.483205,548914,3519758,2000-01-22T00:00:00,None,None,None,00-00148,POINT (548913.472 3519758.907),2000.0,"[22582.678623804346, [110679]]",110679,None,relation,110679,"{'admin_level': '8', 'border_type': 'city', 'b...","MULTIPOLYGON (((552311.153 3492250.219, 552338...",238,United States of America,USA,Sierra Vista,town,NaN,Q79891,"en:Sierra Vista, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TIGER/Line® 2008 Place Shapefiles (http://www....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,city,administrative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boundary
2,25608.503919,[5606091],00-00189,Unidentified,male,,2000-01-20,US Forest Service,,"physical description with directions, distance...",5,Nogales,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,SANTA CRUZ,31.568000,-111.151300,485643,3492563,2000-01-20T00:00:00,None,None,None,00-00189,POINT (485642.300 3492563.305),2000.0,"[25608.50391883214, [5606091]]",5606091,None,relation,5606091,"{'admin_level': '8', 'border_type': 'city', 'b...","MULTIPOLYGON (((502192.255 3468428.516, 502189...",238,United States of America,USA,Nogales,city,NaN,Q79783,"en:Nogales, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TIGER/Line® 2008 Place Shapefiles (http://www....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [77]:


def unpack_meters2miles(row):
    return row['road_distance'][0]/1609.34

deaths_towns_merged['road_distance_miles'] = deaths_towns_merged.apply(lambda row : unpack_meters2miles(row),axis=1)

deaths_towns_merged

,distance,IDs,ML Number,Name,Sex,Age,Reporting Date,Surface Management,Location,Location Precision,Corridor Code,Corridor,Cause of Death,OME Determined COD,Condition Code,Body Condition,Post Mortem Interval,State,County,Latitude,Longitude,UTM X,UTM Y,Date,OME number,YCSO number,Nationality,merge_id,geometry_x,Reporting_year,road_distance,unpacked_ID,unpacked_ID_2,type_x,id,tags,geometry_y,index_right,ADMIN,ISO_A3,name,place,population,wikidata,wikipedia,census:population,ele,gnis:County,gnis:County_num,gnis:ST_alpha,gnis:ST_num,gnis:id,import_uuid,is_in,gnis:Class,name:uk,gnis:feature_id,population:date,website,name:en,name:ru,population:source,source,alt_name,official_name,founded,incorporated,start_date,name:ar,name:he,alt_name:es,capital,layer,name:da,name:eo,name:es,name:hak,name:haw,name:hi,name:ja,name:kn,name:ko,name:lt,name:lv,name:nan,name:nv,name:oc,name:pl,name:pt,name:ta,name:zh,name:zh-Hans,name:zh-Hant,source:name:oc,state_capital,fixme,addr:postcode,admin_level,border_type,boundary,is_in:country,is_in:country_code,is_in:iso_3166_2,is_in:state,is_in:state_code,tiger:CLASSFP,tiger:CPI,tiger:FUNCSTAT,tiger:LSAD,tiger:MTFCC,tiger:NAME,tiger:NAMELSAD,tiger:PCICBSA,tiger:PCINECTA,tiger:PLACEFP,tiger:PLACENS,tiger:PLCIDFP,tiger:STATEFP,tiger:reviewed,type_y,road_distance_miles
0,7381.303873,[2590624],00-00097,Unidentified,male,,2000-01-10,Cocopah Indian Reservation,,"physical description with directions, distance...",12,Yuma,Skeletal Remains,UNDETERMINED - MUMMIFIED AND SKELETAL REMAINS,4,Skeletonization w/ mummification,< 5 weeks,Arizona,YUMA,32.589668,-114.799807,143294,3612175,2000-01-10T00:00:00,None,None,None,00-00097,POINT (143293.359 3612175.618),2000.0,"[7381.303872986647, [2590624]]",2590624,None,relation,2590624,"{'admin_level': '8', 'border_type': 'city', 'b...","MULTIPOLYGON (((141619.197 3601615.589, 141590...",238,United States of America,USA,San Luis,city,NaN,Q80018,"en:San Luis, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"USA, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,city,administrative,USA,US,US:AZ,Arizona,AZ,C1,Y,A,25,G4110,San Luis,San Luis city,N,N,63470,02411795,0463470,04,no,boundary,4.586541
1,22582.678624,[110679],00-00148,"Nicolas, Tomas",male,17,2000-01-22,State of Arizona,,"physical description with directions, distance...",4,Patagonia,Exposure,HYPOTHERMIA,1,Fully fleshed,< 1 day,Arizona,PIMA,31.812406,-110.483205,548914,3519758,2000-01-22T00:00:00,None,None,None,00-00148,POINT (548913.472 3519758.907),2000.0,"[22582.678623804346, [110679]]",110679,None,relation,110679,"{'admin_level': '8', 'border_type': 'city', 'b...","MULTIPOLYGON (((552311.153 3492250.219, 552338...",238,United States of America,USA,Sierra Vista,town,NaN,Q79891,"en:Sierra Vista, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TIGER/Line® 2008 Place Shapefiles (http://www....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,city,administrative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,boundary,14.032261
2,25608.503919,[5606091],00-00189,Unidentified,male,,2000-01-20,US Forest Service,,"physical description with directions, distance...",5,Nogales,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,SANTA CRUZ,31.568000,-111.151300,485643,3492563,2000-01-20T00:00:00,None,None,None,00-00189,POINT (485642.300 3492563.305),2000.0,"[25608.50391883214, [5606091]]",5606091,None,relation,5606091,"{'admin_level': '8', 'border_type': 'city', 'b...","MULTIPOLYGON (((502192.255 3468428.516, 502189...",238,United States of America,USA,Nogales,city,NaN,Q79783,"en:Nogales, Arizona",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TIGER/Line® 2008 Place Shapefiles (http:/

In [78]:
deaths_towns_merged.to_csv('../migrant_deaths_analysis/distance_from_roads_towns_border/towns/deaths_towns.csv')

